<center><img src="MKn_Staffelter_Hof.jpeg" alt="Picture of old business"</center>
<!--Image Credit: Martin Kraft https://commons.wikimedia.org/wiki/File:MKn_Staffelter_Hof.jpg -->

Staffelter Hof Winery is Germany's oldest business, established in 862 under the Carolingian dynasty. It has continued to serve customers through dramatic changes in Europe, such as the Holy Roman Empire, the Ottoman Empire, and both world wars. What characteristics enable a business to stand the test of time?

To help answer this question, BusinessFinancing.co.uk researched the oldest company still in business in **almost** every country and compiled the results into several CSV files. This dataset has been cleaned.

Having useful information in different files is a common problem. While it's better to keep different types of data separate for data storage, you'll want all the data in one place for analysis. You'll use joining and data manipulation to work with this data and better understand the world's oldest businesses.

## The Data
`data/businesses.csv` and `data/new_businesses.csv`
|Column|Description|
|------|-----------|
|`business`|Name of the business (varchar)|
|`year_founded`|Year the business was founded (int)|
|`category_code`|Code for the business category (varchar)|
|`country_code`|ISO 3166-1 three-letter country code (char)|

`data/countries.csv`
|Column|Description|
|------|-----------|
|`country_code`|ISO 3166-1 three-letter country code (varchar)|
|`country`|Name of the country (varchar)|
|`continent`|Name of the continent the country exists in (varchar)|

`data/categories.csv`
|Column|Description|
|------|-----------|
|`category_code`|Code for the business category (varchar)|
|`category`|Description of the business category (varchar)|

In [40]:
# Import necessary libraries
import pandas as pd

# Load the data
businesses = pd.read_csv("data/businesses.csv")
new_businesses = pd.read_csv("data/new_businesses.csv")
countries = pd.read_csv("data/countries.csv")
categories = pd.read_csv("data/categories.csv")

In [41]:
# Merge dfs businesses and countries
bus_coun = pd.merge(businesses,countries,how="inner",on="country_code")
bus_coun.head

# Group by contient and finding the oldest business by continent using min in year founded
continent = bus_coun.groupby("continent").agg({"year_founded":"min"})

#merge bus_coun with continent 
merged_continent = pd.merge(bus_coun,continent,on=["continent","year_founded"])

#Filter the df to obtain only the required columns and print the result
oldest_business_continent = merged_continent[["continent","country","business","year_founded"]]
print(oldest_business_continent)

# Concatenate new bussinesses with businesses and merge with countries df
new_buss_df = pd.concat([businesses,new_businesses])
new_coun_df = pd.merge(new_buss_df,countries,how="outer",on="country_code",indicator=True)
print(new_coun_df.head())

# Filter the df for finding countries with missing business data
missing_countries = new_coun_df[new_coun_df["_merge"] != "both"]

# Group by to count the amount of countries per continent with missing data. Rename the column of the df to count_missing
count_missing = missing_countries.groupby("continent").agg({"country":"count"})
count_missing.columns = ["count_missing"]
print(count_missing)

# Merge businesses with categories and countries
busines_cat = pd.merge(businesses,categories,how="inner",on="category_code")
business_cat_coun = pd.merge(busines_cat,countries,how="inner",on="country_code")

# Group the df in order to obtain the oldest business for each continent and category
oldest_by_continent_category = business_cat_coun.groupby(["continent","category"]).agg({"year_founded":"min"})
print(oldest_by_continent_category)


       continent    country                     business  year_founded
0         Africa  Mauritius               Mauritius Post          1772
1           Asia      Japan                   Kongō Gumi           578
2         Europe    Austria  St. Peter Stifts Kulinarium           803
3  North America     Mexico  La Casa de Moneda de México          1534
4  South America       Peru      Casa Nacional de Moneda          1565
5        Oceania  Australia               Australia Post          1809
                         business  year_founded  ... continent _merge
0                  Hamoud Boualem        1878.0  ...    Africa   both
1  Communauté Électrique du Bénin        1968.0  ...    Africa   both
2        Botswana Meat Commission        1965.0  ...    Africa   both
3                     Air Burkina        1967.0  ...    Africa   both
4                         Brarudi        1955.0  ...    Africa   both

[5 rows x 7 columns]
               count_missing
continent                   
Afr